In [76]:
!pip install --upgrade pip
!pip install pymongo
!pip install -U nltk
!pip install joblib

Requirement already up-to-date: pip in /opt/conda/lib/python3.6/site-packages (18.1)
    100% |████████████████████████████████| 1.4MB 4.1MB/s 
  Running setup.py bdist_wheel for nltk ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/4b/c8/24/b2343664bcceb7147efeb21c0b23703a05b23fcfeaceaa2a1e
Successfully built nltk
    100% |████████████████████████████████| 286kB 9.2MB/s 


In [144]:
import pymongo
import pandas as pd
import re
import nltk

from nltk.tokenize import TreebankWordTokenizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from joblib import dump, load

#Download do corpus da nltk
#nltk.download()

In [65]:
def _connect_mongo(host, port, username, password, db):
    """ A util for making a connection to mongo """

    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)


    return conn[db]

In [66]:
def read_mongo(db, collection, query={}, host='ds249824.mlab.com', port='49824', username='app', password='nodeapp01', no_id=True):
    """ Read from Mongo and Store into DataFrame """

    # Connect to MongoDB
    db = _connect_mongo(host=host, port=port, username=username, password=password, db=db)

    # Make a query to the specific DB and Collection
    cursor = db[collection].find(query)

    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))

    # Delete the _id
    if no_id:
        del df['_id']

    return df

In [71]:
acidente = read_mongo('labeling_zone','tweets',query={'label':'Sim'})
nao_acidente = read_mongo('labeling_zone','tweets',query={'label':'Não'})
full_data = acidente.append(nao_acidente)

full_data.head()

,data,id,label,labeled_by,texto,usuario
0,Sun Dec 16 11:27:33 +0000 2018,1074264765747335169,Sim,NaN,"""""""Cidadão de bem"""""" confessou ter matado duas...",AIice_Costa
1,Sat Dec 29 10:22:21 +0000 2018,1078959397752266753,Sim,NaN,"""#INFO #RDRJ temporealnews RT bandnewsfmrio: P...",TempoRealNews
2,Fri Dec 28 10:22:18 +0000 2018,1078596997848928257,Sim,NaN,"""#RDRJ RT OperacoesRio: AV. BRASIL | BONSUCESS...",TempoRealNews
3,Thu Dec 27 10:03:12 +0000 2018,1078229806150496256,Sim,NaN,"""#RDRJ RT OperacoesRio: CAMINHO PARA RODOVIÁRI...",TempoRealNews
4,Sat Dec 29 09:52:11 +0000 2018,1078951805688705024,Sim,NaN,"""#RDRJ RT OperacoesRio: LINHA VERMELHA - Senti...",TempoRealNews


In [70]:
def remove_urls(text):
    return re.sub(r'https?:\/\/.*[\r\n]*', '', text)

def to_lower(text):
    return text.lower()

In [74]:
full_data['texto_formatado'] = full_data['texto'].apply(lambda t: remove_urls(str(t)))
full_data['texto_formatado'] = full_data['texto_formatado'].apply(lambda t: to_lower(t))

In [79]:
tokenizer = TreebankWordTokenizer()
full_data['tokens'] = full_data['texto_formatado'].apply(lambda t: tokenizer.tokenize(t))
full_data[['texto_formatado','tokens']].head()

,texto_formatado,tokens
0,"""""""cidadão de bem"""""" confessou ter matado duas...","[``, ``, '', cidadão, de, bem, '', '', '', con..."
1,"""#info #rdrj temporealnews rt bandnewsfmrio: p...","[``, #, info, #, rdrj, temporealnews, rt, band..."
2,"""#rdrj rt operacoesrio: av. brasil | bonsucess...","[``, #, rdrj, rt, operacoesrio, :, av., brasil..."
3,"""#rdrj rt operacoesrio: caminho para rodoviári...","[``, #, rdrj, rt, operacoesrio, :, caminho, pa..."
4,"""#rdrj rt operacoesrio: linha vermelha - senti...","[``, #, rdrj, rt, operacoesrio, :, linha, verm..."


In [135]:
def create_stopword_list():
    portuguese_stops = set(stopwords.words('portuguese'))
    portuguese_stops.add('rt')

    with open('punctuation.txt','r+') as punct_file:
        puncts = punct_file.readlines()

    for item in puncts:    
        portuguese_stops.add(item.strip())
        
    return portuguese_stops

stop_w = create_stopword_list()

#aplica a remocao de stop-words
full_data['words'] = full_data['tokens'].apply(lambda w: [word for word in w if word not in stop_w]) 
#exibe resultado intermediario
full_data[['tokens','words']].head(10)

,tokens,words
0,"[``, ``, '', cidadão, de, bem, '', '', '', con...","[cidadão, bem, confessou, ter, matado, duas, t..."
1,"[``, #, info, #, rdrj, temporealnews, rt, band...","[info, rdrj, temporealnews, bandnewsfmrio, cau..."
2,"[``, #, rdrj, rt, operacoesrio, :, av., brasil...","[rdrj, operacoesrio, av., brasil, bonsucesso, ..."
3,"[``, #, rdrj, rt, operacoesrio, :, caminho, pa...","[rdrj, operacoesrio, caminho, rodoviária, trân..."
4,"[``, #, rdrj, rt, operacoesrio, :, linha, verm...","[rdrj, operacoesrio, linha, vermelha, sentido,..."
5,"[``, #, rdrj, rt, operacoesrio, :, tanque, |, ...","[rdrj, operacoesrio, tanque, rua, cândido, ben..."
6,"[``, erro, humano, '', na, origem, do, acident...","[erro, humano, origem, acidente, elétrico, 25...."
7,"[``, gado, na, pista, provoca, acidente, '', k...","[gado, pista, provoca, acidente, kkkkkkkkkkk]"
8,"[``, jovem, perde, a, vida, em, grave, acident...","[jovem, perde, vida, grave, acidente, avenida,..."
9,"[``, manu, ,, n, surta, ,, sofri, um, acidente...","[manu, n, surta, sofri, acidente, moto, quase,..."


In [136]:
stemer= SnowballStemmer(language='portuguese')
full_data['stem_words'] = full_data['words'].apply(lambda t: [stemer.stem(word) for word in t])
full_data[['words','stem_words']].head()

,words,stem_words
0,"[cidadão, bem, confessou, ter, matado, duas, t...","[cidadã, bem, confess, ter, mat, duas, técnic,..."
1,"[info, rdrj, temporealnews, bandnewsfmrio, cau...","[info, rdrj, temporealnews, bandnewsfmri, caus..."
2,"[rdrj, operacoesrio, av., brasil, bonsucesso, ...","[rdrj, operacoesri, av., brasil, bonsucess, ac..."
3,"[rdrj, operacoesrio, caminho, rodoviária, trân...","[rdrj, operacoesri, caminh, rodoviár, trânsit,..."
4,"[rdrj, operacoesrio, linha, vermelha, sentido,...","[rdrj, operacoesri, linh, vermelh, sent, centr..."


In [137]:
def encode_label(text):
    if text=='Sim':
        return 1
    elif text=='Não':
        return 0
    else:
        return -1

full_data['target'] = full_data['label'].apply(lambda label: encode_label(label))
full_data['target'].describe()

count    256.000000
mean       0.417969
std        0.494191
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
Name: target, dtype: float64

In [138]:
full_data['clean_text'] = full_data['stem_words'].apply(lambda t: str(' '.join(t)))

count_vect = CountVectorizer()
X_TF = count_vect.fit_transform(full_data['clean_text'])
print(X_TF.shape)

(256, 1592)


In [159]:
print(count_vect.get_feature_names())

['01', '027998504942', '040', '050', '085', '101', '106', '10h00', '10ºgbm', '116', '12', '12h15', '12h35', '13h03', '140', '14h02', '15', '150', '15h40', '16', '17', '17h19', '18', '1937', '1ª', '20', '2008', '2018', '2018emfot', '2019', '20h35', '210', '23', '232', '24', '245', '25', '250', '258', '259', '26', '262', '27', '28', '29', '290', '294', '2ª', '30', '35', '376', '381', '386', '3997', '3h', '401', '40h', '447', '470', '50', '525', '535', '603', '61', '614', '64', '667', '67', '676', '70', '75', '751', '76', '826', '91', '_______________________________', '_eco101', 'aaaaaa', 'abaix', 'abaixo', 'abert', 'abord', 'absolut', 'absurd', 'acab', 'acas', 'acertou', 'acess', 'acha', 'acham', 'acho', 'achou', 'acident', 'acidental', 'acidente', 'acidentes', 'acionada', 'acompanh', 'acontec', 'aconteceu', 'acord', 'acredit', 'aeroporto', 'afins', 'agent', 'agor', 'agr', 'aguard', 'aguent', 'agênc', 'agênciagbc', 'ah', 'ai', 'ain', 'aind', 'ajam', 'ajud', 'alag', 'alago', 'alain', 'al

In [160]:
print(X_TF.toarray()) 

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


In [162]:
print(.get_stop_words())

AttributeError: get_stop_words not found

In [155]:
tfidf_transformer = TfidfTransformer()
X = tfidf_transformer.fit_transform(X_TF)
X.shape

(256, 1592)

In [140]:
X_train, X_test, y_train, y_test = train_test_split(X, full_data['target'], test_size=0.33, random_state=42)

In [145]:
nb = MultinomialNB().fit(X_train, y_train)
svm = svm.SVC().fit(X_train,y_train)
tree = tree.DecisionTreeClassifier().fit(X_train,y_train)
logr = LogisticRegression().fit(X_train,y_train)

In [151]:
def print_metrics(y_true,y_predicted):
    acc = accuracy_score(y_test.values,y_predicted)
    print('Acurácia: ' + str(acc))
    #accuracy_score(y_true, y_predicted)
    print('Matriz de Confusão:' )
    print(confusion_matrix(y_true, y_predicted, labels=[0, 1]))

In [152]:
print('Naive Bayes')
print_metrics(y_test,nb.predict(X_test))
print('\n')
print('Decision Tree')
print_metrics(y_test,tree.predict(X_test))
print('\n')
print('Suport Vector Machines')
print_metrics(y_test,svm.predict(X_test))
print('\n')
print('Logistic Regressor')
print_metrics(y_test,logr.predict(X_test))

Naive Bayes
Acurácia: 0.870588235294
Matriz de Confusão:
[[45  7]
 [ 4 29]]


Decision Tree
Acurácia: 0.823529411765
Matriz de Confusão:
[[45  7]
 [ 8 25]]


Suport Vector Machines
Acurácia: 0.611764705882
Matriz de Confusão:
[[52  0]
 [33  0]]


Logistic Regressor
Acurácia: 0.811764705882
Matriz de Confusão:
[[50  2]
 [14 19]]
